## 1. Farmers' Market Expansion
<p>You have been hired by a farming organisation that helps local farmers sell their products. They want to know whether they should open up a new farmers' market to sell dairy products from nearby farmers. They have supplied you with daily shopping data from a panel of local households from 2019 to 2020. </p>
<p>The organization will make their decision based on whether dairy products are popular in the area, and whether sales are trending in a positive direction. To answer these questions, they want three pieces of data:</p>
<ol>
<li>What was the total number of purchases of dairy products for each month of 2020 (i.e., the <code>total_sales</code>)?</li>
<li>What was the total share of dairy products (out of all products purchased) for each month of 2020 (i.e., the <code>market_share</code>)?</li>
<li>For each month of 2020, what was the percentage increase or decrease in total monthly dairy purchases compared to the same month in 2019 (i.e., the <code>year_change</code>)?</li>
</ol>
<p>The organization handles not only dairy farmers, but also those with chicken farms. As a result, they are only interested in these three categories (which they treat as dairy): ‘whole milk’, 'yogurt' and 'domestic eggs'.</p>
<p>The data you need is available in the tables shown in the database schema below.</p>
<h5 id="databaseschema">Database Schema</h5>
<p><img src="https://assets.datacamp.com/production/repositories/5960/datasets/463543c8c38957ca5b95d93b02f2cb1bec53334f/diagram.PNG" alt="Database Schema" width="400px"></p>

In [ ]:
%%sql
postgresql:///groceries
    


WITH
T_2019 AS (SELECT 
                    month,
                    year,
                    category
             FROM
                    purchases_2019
               LEFT JOIN  categories
               USING(purchase_id)), 

T_2020 AS (SELECT CAST( date_part('month', TO_DATE(fulldate, 'YYYY/MM/DD')) AS INT) as month,
                  CAST( date_part('year', TO_DATE(fulldate, 'YYYY/MM/DD')) AS INT) as year,
                  category 
                FROM
                      purchases_2020
              LEFT JOIN  categories as c
               ON c.purchase_id = purchases_2020.purchaseid),
D_2019 AS ( SELECT 
                   month,
                   count(*)*1.0 as count_2019
            FROM 
              T_2019
              WHERE 
              category IN ('whole milk', 'yogurt' , 'domestic eggs')
              GROUP BY 1
),
D_2020 AS ( SELECT 
                   month,
                   count(*)*1.0 as count_2020
            FROM 
              T_2020
              WHERE 
              category IN ('whole milk', 'yogurt' , 'domestic eggs')
              GROUP BY 1
), 
full_data AS (SELECT *
FROM D_2020
LEFT JOIN D_2019
USING (month))

SELECT month, 
       count_2020 as total_sales,
       ROUND ((count_2020 / (SELECT COUNT(*)
                    FROM T_2020 
                    WHERE full_data.month = T_2020.month) * 100.0),2) as market_share,
        ROUND (((count_2020 - count_2019)/count_2019)*100.0,2 ) as year_change
FROM full_data 
ORDER BY 1



